In [1]:
import cv2 
import numpy as np

C:\Users\Bikal\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Bikal\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\Bikal\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [8]:
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import pandas as pd

class LKTracker:
    def __init__(self):
        global lk_box_width, lk_box_height
        
        self.params = dict(winSize = (lk_box_width, lk_box_height), maxLevel = 5, criteria = 
                           (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))      
        self.crop = None        
        self.old_points = None
        self.color = None
        self.width = None
        self.height = None
        self.active = False
        self.inactive_frames = 0
        
    def set_width_height(self, width, height):
        self.width = int(width * 0.9)
        self.height = int(height // 2)
        
    def track(self, old_gray, curr_gray):     
        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, curr_gray, self.old_points, None, **self.params)
        self.maintain_active(new_points)
        return new_points  
    
    def maintain_active(self, new_points):
        global keep_alive 
        
        if dist(new_points.ravel(), self.old_points.ravel()) < 2:    
            self.inactive_frames += 1
        else:
            self.inactive_frames = 0
            
        if self.inactive_frames > keep_alive:
            self.active = False    
            
def find_closest(trackers, point, closest_threshold):
    bestKey = -1
    for key, tracker in trackers.items():               
        temp = dist((point[0], point[1]), tracker.old_points.ravel())     
        if temp < closest_threshold:
            closest_threshold = temp
            bestKey = key            
    return bestKey
        
# Codes provided by the professor
def dist(p1, p2):
    """ Returns the distance between two points"""
    return ( (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 ) ** (0.5)

def count_objects(frame, count, winname):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 255, 0)
    thickness = 2

    frame = cv2.putText(frame, 'Total num. of objects seen:'+str(count), org, font,
                        fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow(winname,frame)
    
def add_objects(height, width, trackers, column, row):
    global object_id, frame, colors
    
    object_id = object_id + 1
    tracker = LKTracker()
    tracker.set_width_height(height, width)
    tracker.old_points = np.float32([[column  , row ]])
    tracker.color = colors.pop(0)
    tracker.active = True
    trackers[object_id] = tracker
    
    return trackers
    
def detect_face_and_add_trackers(object_cascade, binary, trackers, scale_factor, min_neighbors, closest_threshold):
   
    global object_id, frame
    detected_faces = object_cascade.detectMultiScale(binary, scale_factor, min_neighbors)
        
    crops = []

    for (column, row, width, height) in detected_faces:              
        crop = binary[ row : row + height, column : column + width]

        crops.append(crop)
        if (height > 20 and width > 20 and height < 200 and width < 200):         
            if len(trackers) == 0:
                trackers = add_objects(height, width, trackers, column, row)
            else:                
                closest = find_closest(trackers, (column, row), closest_threshold)
                
                if closest == -1:
                    trackers = add_objects(height, width, trackers, column, row)
                else:
                    trackers[closest].old_points = np.float32([[column , row ]])                                    
    return trackers

def calculate_metric(metric_cal, frame_count, tracker, f_id, x, y):
    metric_cal = metric_cal.append({'frame': frame_count,'id' : f_id, 
                           'bl': x,'bt' : y,'bw': tracker.width,'bh': tracker.height,
                           'conf' : -1,'x' : -1, 'y' : -1, 'z' : -1}, ignore_index=True)
    return metric_cal

def save_file(metric_cal):
    numpy_array = metric_cal.to_numpy()
    np.savetxt("PETS09-S2L1.txt", numpy_array, fmt = "%d",  delimiter=',')

In [9]:
import numpy as np
import cv2
import sys
from scipy.spatial import distance


def run(image_size, scale_factor, min_neighbors, closest_threshold, vid_path, frame_skip):
    global object_id, frame_rate_para, metric_cal
    
    # resize dimensions can be changed here
    dim = (imagewidth, imageheight)
    font = cv2.FONT_HERSHEY_SIMPLEX

    # The default capture device is the default video source.
    cap = None
    cap = cv2.VideoCapture(vid_path)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_fullbody.xml")

    # Create a window
    winname = "Faces"
    cv2.namedWindow(winname, cv2.WINDOW_NORMAL)

    hasframe, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    old_gray = gray
   

    # Create some random colors
 

    trackers = {}
    frame_count = 1

    

    while(hasframe):    
        hasframe, frame = cap.read()    

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if frame_count % frame_skip == 0:
            trackers = detect_face_and_add_trackers(face_cascade, gray, trackers, scale_factor, min_neighbors, closest_threshold)

        for key, tracker in trackers.items():     

            if tracker.active == True:
                new_point = tracker.track(old_gray, gray)  
                x,y = new_point.ravel()                 
                tracker.old_points = new_point
                x,y = int(x), int(y)

                color = (int(tracker.color[0]),int(tracker.color[1]),int(tracker.color[2]))                     
                cv2.rectangle(frame, (x - tracker.height, y - tracker.width), (x + tracker.height, y + tracker.width), tuple(color), 2)                        
                cv2.putText(frame, str(key), (x - 10, y), font, 1,  tuple(color), 2)

                metric_cal = calculate_metric(metric_cal, frame_count, tracker,key, x, y)

        if frame_count % 100 == 0:
            save_file(metric_cal)

        count_objects(frame, len(trackers), winname)
        cv2.imshow(winname, frame)

        old_gray = gray

        key = cv2.waitKey(frame_rate_para)
        if key == 27:
            break
        frame_count = frame_count + 1

    save_file(metric_cal)
    cap.release()
    cv2.destroyAllWindows()

In [10]:
metric_cal = pd.DataFrame(columns = ['frame','id','bl','bt','bw','bh', 'conf','x', 'y', 'z'])
colors = list(np.random.randint(0,255,(1000,3)))
colors = [tuple(x) for x in colors]
    

imagewidth = 500
imageheight = 1000
scale_factor = 1.1
min_neighbors = 2
closest_threshold = 50
vid_path = 'PETS09-S2L1.mp4'
frame_skip = 2
object_id = 0
keep_alive = 10
frame_rate_para = 5

    
lk_box_width = 25
lk_box_height = 75

image_size = (imagewidth, imageheight)


run(image_size,  scale_factor, min_neighbors, closest_threshold, vid_path, frame_skip)

In [330]:
trackers = {}

# metric_cal

In [ ]:
numpy_array = metric_cal.to_numpy()
np.savetxt("test_file.txt", numpy_array, fmt = "%d",  delimiter=',')

In [394]:
metric_cal

,frame,id,bl,bt,bw,bh,conf,x,y,z
0,2,1,506,420,61,17,-1,-1,-1,-1
1,2,2,490,147,73,20,-1,-1,-1,-1
2,3,1,506,419,61,17,-1,-1,-1,-1
3,3,2,488,147,73,20,-1,-1,-1,-1
4,4,1,507,426,61,17,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
533,165,22,514,211,99,27,-1,-1,-1,-1
534,166,9,652,145,54,15,-1,-1,-1,-1
535,166,19,516,256,86,24,-1,-1,-1,-1
536,166,20,515,252,81,22,-1,-1,-1,-1
